In [1]:
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request
import requests
import pandas as pd
import re
import csv

def crawl_page(link=''):
    # req = urllib.request.Request(
    #         link, 
    #         data=None, 
    #         headers={
    #             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    #         }
    # )
    # try:
    #     page= urllib.request.urlopen(req)
    #     return_code= page.getcode()
    #     soup_page= BeautifulSoup(page,'html.parser')
    #     return {'page':soup_page, 'status': return_code}
    # except HTTPError as err:
    #     if err.code == 404:
    #         return_code=404
    #         return {'page':'no page', 'status': return_code}
    #     else:
    #         raise
    try:
        response = requests.get(
            url="http://api.scraperapi.com", #scrapeapi
            params={
                'api_key': 'e3458ff579ef5c1369f185af0ceaecfb',
                'url': link,  
            },
            
        )
        return_code= 200
        soup_page= BeautifulSoup(response.content,'html.parser')
        return {'page':soup_page, 'status': return_code}
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            return_code=404
            return {'page':'no page', 'status': return_code}
    

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

def remove_row_csv():
    with open('temp_27.csv') as inp:
        for link in csv.reader(inp):
            count=0
            temp=link
            with open('used_estate.txt') as out:
                for estate in csv.reader(out):
                    if re.match("(.*)/"+estate[0],temp[1]) is not None:
                        count+=1
            if count==0:
                new_row= [link[0]+','+link[1]]
                write_csv(new_row,"temp_28.csv")

def create_urls(file=''):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        for row in csv_reader:
            new_row= [str(line_count)+','+row[0]]
            write_csv(new_row,'property_url_v4.csv')
            line_count+=1

    

In [8]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)['page']
        self.__area_url=[]
        self.__list_url=[]
        self.__urls=pd.read_csv("temp_43_v4.csv",usecols= ['id','url']) 
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)['page']
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)['page']
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        data={}

        estate_id= link.split('/')[-1].split('?')[0]

        prop_soup_element= crawl_page(self.__url_to_open+link)

        
        if prop_soup_element['status'] !=200:
            print('heres')
            return False
        
        prop_soup= prop_soup_element['page']

        if prop_soup is None:
            print('here')
            return False

        #check if page not found
        check_page_not_found_msg= prop_soup.select_one('.p24_not_found')
        if check_page_not_found_msg is not None:
            print("====> Page not found !!")
            return False
        
        #check page expired
        check_page_expired = prop_soup.select_one('.p24_results.p24_expired')
        if check_page_expired is not None:
            print("====> Page expired !!")
            return False
        

        name= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('h1').get_text()
        data['name']=name

        price= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('div',class_="p24_price").get_text()
        data['price']=price

        province= link.split('/')[4]
        data['province']=province

        city= link.split('/')[3]
        data['city']=city


        is_adress= len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) >= 3
        if is_adress:
            if len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) == 3:
                index=-1
            else:
                index=3
            address= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")[index].get_text()
            data['address']=address
        else:
            address=  prop_soup.select_one('.p24_addressPropOverview').get_text()
            data['address']=address

        #property overview
        if type(prop_soup.select_one('.p24_propertyOverview')) is not None:
            prop_overview_panel= prop_soup.find('div',class_='p24_propertyOverview').find('div',class_='panel-body')
            prop_overview_row= prop_overview_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #rooms
        rooms_panel=prop_soup.select_one('#js_accordion_rooms')
        if rooms_panel is not None:
            prop_overview_row= rooms_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #external_features
        external_features_panel=prop_soup.select_one('#js_accordion_externalfeatures')
        if external_features_panel is not None:
            prop_overview_row= external_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value
        
        #other_features
        other_features_panel=prop_soup.select_one('#js_accordion_otherfeatures')
        if other_features_panel is not None:
            prop_overview_row= other_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #building_features
        building_features_panel=prop_soup.select_one('#js_accordion_building')
        if building_features_panel is not None:
            prop_overview_row= building_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #points_interest
        interest_soup= crawl_page(self.__url_to_open+'/ListingReadOnly/PointsOfInterestForListing?ListingNumber=P24-'+estate_id)
        # points_interest_panel=prop_soup.select_one('#accordian-points-of-interest')
        # print(points_interest_panel)
        if interest_soup['status'] == 200:
            categories= interest_soup['page'].select('.js_P24_POICategory')
            for cat in categories:
                interest= cat.select_one('.row > .col-6 > .p24_semibold')
                distances= cat.select('.row > .col-6.noPadding.p24_semibold')
                dist_tab=[]
                for dist in distances:
                    dist_tab.append(dist.get_text())
                data[interest.get_text()]=dist_tab

        print(str(data))
        write_csv([str(data)],'data_property_v4.csv')

        
        return True

    def crawl_property_per_page(self,city,page,number_properties):
        link=self.__url_to_open+city+'/p'+str(page)
        print('==>Page link: ',link)
        cur_soup= crawl_page(link)['page']
        properties= cur_soup.find_all('div',class_='js_resultTile')
        for prop_index in range(len(properties)-1):
            prop_link= properties[prop_index].find('a').get('href')
            write_csv([prop_link],'data_property_url.csv')

        number_properties+= len(properties)
        return number_properties

    def crawl_property_per_city(self,city,cur_page):
        soup= crawl_page(self.__url_to_open+city)['page']
        pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
        page_number= int(pagination)
        number_properties=0
        print("City url: ",city)
        for page in range(cur_page, page_number+1):
            print("=>Page number: ",page)
            number_properties = self.crawl_property_per_page(city,page,number_properties)
        print(number_properties)
    
    def crawl_properties(self):
        #shuffle
        self.__urls = self.__urls.sample(frac = 1).reset_index(drop=True)
        for i in range(self.__urls.size):
            prop_link= self.__urls.iloc[i]['url']
            
            print('==>Property link: ', prop_link)
            check_property_crawl = self.get_property_data(prop_link)
            if check_property_crawl:
                self.__urls= self.__urls.drop(self.__urls.index[i])
                self.__urls.to_csv('temp_44_v4.csv',index=False)

    def crawl_property_url(self):
        self.__list_url=['/townhouses-for-sale/gauteng/1']
        for link in self.__list_url:
            self.crawl_property_per_city(link,1)

In [9]:
dc= DataCollect()
try:
    dc.crawl_properties()
except:
    dc.crawl_properties()

==>Property link:  /for-sale/jackaroo-park/witbank/mpumalanga/2518/109766672
{'name': '3 Bedroom House for Sale in Jackaroo Park', 'price': 'R 779 000', 'province': 'mpumalanga', 'city': 'witbank', 'address': 'Jackaroo Park, Witbank', 'Listing Number': '109766672', 'Type of Property': 'House', 'Listing Date': '18 March 2021', 'Erf Size': '350 m²', 'Floor Size': '129 m²', 'Levies': 'R 608', 'Special Levy': 'R 0', 'Rates and Taxes': 'R 500', 'Pets Allowed': 'Yes', 'Coverage': '36 %', 'Bedrooms': '3', 'Bathroom': 'Full, Basin, Bath, Shower, Toilet', 'Kitchens': '1', 'Lounges': '1', 'Garage': '1', 'Education': ['2.42km', '4.15km', '4.24km', '4.50km', '4.72km', '5.09km', '5.39km', '5.47km', '5.62km', '5.64km', '5.88km', '5.92km', '6.06km', '6.06km', '7.00km', '7.11km', '7.39km', '7.43km', '7.53km', '7.55km', '7.56km', '7.60km', '7.64km', '7.68km', '7.71km', '7.76km'], 'Sports and Leisure': ['5.18km'], 'Shopping': ['6.49km', '8.09km'], 'Food and Entertainment': ['8.08km']}
==>Property link: 

{'name': '2 Bedroom Apartment / Flat for Sale in Randpark Ridge', 'price': 'R 899 000', 'province': 'gauteng', 'city': 'randburg', 'address': 'Randpark Ridge, Randburg', 'Listing Number': '109721627', 'Type of Property': 'Apartment / Flat', 'Description': 'First Floor', 'Listing Date': '08 March 2021', 'Pets Allowed': 'No', 'Bedrooms': '2', 'Bedroom 1': ' ', 'Bedroom 2': ' ', 'Bathrooms': '2', 'Kitchen': ' ', 'Lounge': ' ', 'Parking': '1', 'Pool': 'Yes', 'Security': 'Electric fencing', 'Education': ['0.37km', '0.59km', '0.59km'], 'Food and Entertainment': ['0.45km', '0.53km', '0.58km', '0.62km', '0.75km', '0.75km', '0.85km', '0.86km', '0.87km', '0.88km', '0.88km', '1.10km', '1.24km', '1.24km', '1.24km', '1.25km', '1.26km'], 'Transport and Public Services': ['0.57km', '0.73km', '1.29km', '1.29km'], 'Shopping': ['0.70km', '0.72km', '0.76km'], 'Health': ['0.70km', '1.14km', '1.23km']}
==>Property link:  /for-sale/three-rivers/vereeniging/gauteng/3754/109398845
{'name': '3 Bedroom Townhous

{'name': '4 Bedroom House for Sale in Vaal Marina', 'price': 'R 1 500 000', 'province': 'gauteng', 'city': 'vaal-marina', 'address': '\n4 Bedroom House for Sale in Vaal Marina\n', 'Listing Number': '109706787', 'Type of Property': 'House', 'Street Address': ' 38 Kreef Road, Vaal Marina', 'Listing Date': '03 March 2021', 'Erf Size': '1 088 m²', 'Rates and Taxes': 'R 800', 'Pets Allowed': 'Yes', 'Bedrooms': '4', 'Bathrooms': '3', 'Kitchens': '1', 'Dining Rooms': '1', 'Family/TV Rooms': '1', 'Braai Rooms': '1', 'Bars': '1', 'Garage': 'LOTS OF TRELLIE DOORS AND EXCELLENT SECURITY', 'Parking': 'Visitors Parking, Secure Parking, Double', 'Pool': 'Auto Cleaning Equipment, Chlorinator, Splash Pool', 'Security': 'Totally Fenced, Security Gate, Burglar Bars'}
==>Property link:  /for-sale/muizenberg/cape-town/western-cape/9025/109889518
====> Page expired !!
==>Property link:  /for-sale/pinelands/cape-town/western-cape/8017/109757993
{'name': '1 Bedroom Apartment / Flat for Sale in Pinelands', 'p

{'name': '2 Bedroom Apartment / Flat for Sale in Sibaya Precinct', 'price': 'R 3 850 000', 'province': 'kwazulu-natal', 'city': 'umhlanga', 'address': 'Sibaya Precinct, Umhlanga', 'Listing Number': '110075747', 'Type of Property': 'Apartment / Flat', 'Listing Date': '09 June 2021', 'Erf Size': '19 215 m²', 'Floor Size': '92 m²', 'Price per m²': 'R 41 848', 'Levies': 'R 3 400', 'Rates and Taxes': 'R 1 500', 'Pets Allowed': 'No', 'Furnished': 'Yes', 'Zoning': 'General Residential', 'Bedrooms': '2', 'Bathrooms': '2', 'Kitchens': '1', 'Lounges': '1', 'Parking': '2', 'Pool': 'Yes', 'Sports and Leisure': ['2.80km', '5.18km'], 'Education': ['2.99km', '3.67km', '3.82km', '4.43km', '4.80km', '4.91km', '4.93km'], 'Food and Entertainment': ['3.15km', '4.42km', '5.13km', '5.17km', '5.17km', '5.18km', '5.18km', '5.18km', '5.19km', '5.19km', '5.19km', '5.21km', '5.21km', '5.28km', '5.29km'], 'Health': ['4.81km'], 'Shopping': ['5.17km', '5.17km', '5.28km', '5.28km', '5.29km']}
==>Property link:  /for

====> Page not found !!
==>Property link:  /for-sale/ermelo/ermelo/mpumalanga/4844/109502741
{'name': '3 Bedroom House for Sale in Ermelo', 'price': 'R 1 190 000', 'province': 'mpumalanga', 'city': 'ermelo', 'address': 'Ermelo', 'Listing Number': '109502741', 'Type of Property': 'House', 'Listing Date': '12 January 2021', 'Erf Size': '1 635 m²', 'Bedrooms': '3', 'Bedroom 1': 'Built-In Cupboards, Carpeted Flooring, Curtain Rails', 'Bedroom 2': 'Built-In Cupboards, Carpeted Flooring, Curtain Rails', 'Bedroom 3': 'Built-In Cupboards, Carpeted Flooring, Curtain Rails', 'Bathrooms': '2', 'Bathroom 1': 'Basin, Bath, Shower, Tiled Flooring, Toilet', 'Bathroom 2': 'Basin, Bath, Tiled Flooring, Toilet', 'Kitchen': 'Blinds, Hob, Melamine Kitchen, Tiled Flooring', 'Reception Rooms': '1', 'Dining Room': 'Curtain Rails, Open Plan, Tiled Flooring', 'Family/TV Room': 'Curtain Rails, Open Plan, Satellite Dish, Tiled Flooring', 'Other': 'Tiled Flooring', 'Garage': '2', 'Parking': '2 Garage(s) ', 'Educa

{'name': '6 Bedroom House for Sale in Uvongo', 'price': 'R 3 900 000', 'province': 'kwazulu-natal', 'city': 'margate', 'address': 'Uvongo, Margate', 'Listing Number': '109892663', 'Type of Property': 'House', 'Description': 'Double Storey', 'Lifestyle': 'Coastal/Beach', 'Listing Date': '22 April 2021', 'Erf Size': '1 077 m²', 'Floor Size': '700 m²', 'Rates and Taxes': 'R 1 000', 'Bedrooms': '6', 'Bedroom 1': ' ', 'Bedroom 2': ' ', 'Bedroom 3': ' ', 'Bedroom 4': ' ', 'Bedroom 5': ' ', 'Bedroom 6': ' ', 'Bathrooms': '6', 'Kitchen': ' ', 'Lounges': '2', 'Lounge 1': ' ', 'Lounge 2': ' ', 'Dining Room': ' ', 'Rooms': 'Patio', 'Garage': '2', 'Garage 1': ' ', 'Garage 2': ' ', 'Parking': '2', 'Garden': 'Yes', 'Pool': 'Yes', 'Security': 'Alarm System', 'Flatlet': 'Yes', 'Education': ['0.69km', '3.16km', '3.19km', '3.19km', '3.21km', '4.45km', '5.03km', '5.06km', '5.33km', '5.35km', '5.45km', '5.66km', '6.20km', '6.24km', '6.29km', '6.30km', '6.31km', '6.72km', '7.17km', '7.58km', '7.65km', '7.9

KeyboardInterrupt: 